# VolPy pipeline for processing voltage imaging data 
The processing pipeline includes motion correction, memory mapping, segmentation, denoising and source extraction. The demo shows how to construct the params, MotionCorrect and VOLPY objects and call the relevant functions. 
Dataset courtesy of Karel Svoboda Lab (Janelia Research Campus).

In [1]:

from base64 import b64encode
import cv2
import glob
import h5py
import imageio
from IPython import get_ipython
from IPython.display import HTML, display, clear_output
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

try:
    cv2.setNumThreads(0)
except:
    pass

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
        get_ipython().run_line_magic('matplotlib', 'qt')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.utils.utils import download_demo, download_model
from caiman.source_extraction.volpy import utils
from caiman.source_extraction.volpy.volparams import volparams
from caiman.source_extraction.volpy.volpy import VOLPY
from caiman.source_extraction.volpy.mrcnn import visualize, neurons
import caiman.source_extraction.volpy.mrcnn.model as modellib
from caiman.summary_images import local_correlations_movie_offline
from caiman.summary_images import mean_image
from caiman.paths import caiman_datadir

logfile = None # Replace with a path if you want to log to a file
logger = logging.getLogger('caiman')
# Set to logging.INFO if you want much output, potentially much more output
logger.setLevel(logging.ERROR)
logfmt = logging.Formatter('%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s')
if logfile is not None:
    handler = logging.FileHandler(logfile)
else:
    handler = logging.StreamHandler()
handler.setFormatter(logfmt)
logger.addHandler(handler)


2025-06-18 23:39:57.366932: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-18 23:39:57.366963: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-18 23:39:57.366979: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


## Load demo movie and ROIs

In [13]:
# File path to movie file (will download if not present)
# fnames = './../../../Data/Optosynth/optosynth__1__20__5.tif'
fnames = '/home/janwillem/Data/voltage_HPC2/HPC2/00_03.tif'

# File path to ROIs file (will download if not present)
path_ROIs = '/home/janwillem/Data/voltage_HPC2/HPC2/00_0_mrcnn_ROIs.hdf5'
file_dir = os.path.split(fnames)[0]

# Setup some parameters for data and motion correction dataset parameters
fr = 400                                        # sample rate of the movie
ROIs = None                                     # Region of interests
index = None                                    # index of neurons
weights = None                                  # reuse spatial weights by 
                                                # opts.change_params(params_dict={'weights':vpy.estimates['weights']})
# Motion correction parameters
pw_rigid = False                                # flag for pw-rigid motion correction
gSig_filt = (3, 3)                              # size of filter, in general gSig (see below),
                                                # change this one if algorithm does not work
max_shifts = (5, 5)                             # maximum allowed rigid shift
strides = (48, 48)                              # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)                             # overlap between patches (size of patch strides+overlaps)
max_deviation_rigid = 5                         # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'

opts_dict = {
    'fnames': fnames,
    'fr': fr,
    'index': index,
    'ROIs': ROIs,
    'weights': weights,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan
}


opts = volparams(params_dict=opts_dict)

In [21]:
# Display the movie
# m_orig = cm.load('./../../../Data/voltage_HPC2/HPC2/00_02.tif', subindices=slice(0, 7500))
m_orig = cm.load('/home/janwillem/Data/voltage_HPC2/HPC2/00_03_mc.tif')
print(m_orig.shape)
ds_ratio = 0.2
moviehandle = m_orig.resize(1, 1, ds_ratio)
min_, max_ = np.min(moviehandle), np.max(moviehandle)
moviehandle = cm.movie((moviehandle-min_)/(max_-min_)*255,dtype='uint8')
moviehandle.play(fr=40, q_max=99.5, magnification=4)  # press q to exit

(15000, 116, 498)


In [15]:
# Start a cluster for parallel processing
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=None, single_thread=False)

## Motion Correction

In [16]:
%%time
# Create a motion correction object with the specified parameters
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
mc.motion_correct(save_movie=True)
dview.terminate()

CPU times: user 2.55 s, sys: 525 ms, total: 3.07 s
Wall time: 1min 38s


In [17]:
# Motion correction compared to original movie
m_orig = cm.load(fnames)
m_rig = cm.load(mc.mmap_file)
m_orig.fr = 400
m_rig.fr = 400
ds_ratio = 0.2
moviehandle = cm.concatenate([m_orig.resize(1, 1, ds_ratio) - mc.min_mov * mc.nonneg_movie,
                              m_rig.resize(1, 1, ds_ratio)], axis=2)
min_, max_ = np.min(moviehandle), np.max(moviehandle)
moviehandle = cm.movie((moviehandle-min_)/(max_-min_)*255,dtype='uint8')
moviehandle.play(fr=40, q_max=99.5, magnification=4)  # press q to exit

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.12s/it]


In [ ]:
# Movie subtracted from the baseline
m_rig2 = m_rig.computeDFF(secsWindow=1)[0][:1000]
moviehandle1 = -m_rig2
min_, max_ = np.min(moviehandle1), np.max(moviehandle1)
moviehandle1 = cm.movie((moviehandle1-min_)/(max_-min_)*255,dtype='uint8')
moviehandle1.play(fr=40, q_max=99.5, magnification=4)  # press q to exit

In [18]:
m_rig.save('/home/janwillem/Data/voltage_HPC2/HPC2/00_03_mc.tif')

'/home/janwillem/Data/voltage_HPC2/HPC2/00_03_mc.tif'

In [20]:
from tifffile import imwrite
from tifffile import imread

tif = np.array(imread('/home/janwillem/Data/voltage_HPC2/HPC2/00_03_mc.tif'), dtype=np.float32)
print(tif.max())
imwrite('/home/janwillem/Data/voltage_HPC2/HPC2/00_03_mc.tif', tif)

3227.5383


## Memory Mapping

In [24]:
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=None, single_thread=False)
border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0
fname_new = cm.save_memmap_join(mc.mmap_file, base_name='memmap_',
                           add_to_mov=border_to_0, dview=dview, n_chunks=10)
dview.terminate()

# Change fnames to the new motion corrected one
opts.change_params(params_dict={'fnames': fname_new})    

In [16]:
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=None, single_thread=False)

## Segmentation

In [17]:
# Create mean and correlation images
img = mean_image(mc.mmap_file[0], window = 1000, dview=dview)
img = (img-np.mean(img))/np.std(img)

gaussian_blur = False        # Use gaussian blur when there is too much noise in the video
Cn = local_correlations_movie_offline(mc.mmap_file[0], fr=fr, window=fr*4, 
                                      stride=fr*4, winSize_baseline=fr, 
                                      remove_baseline=True, gaussian_blur=gaussian_blur,
                                      dview=dview).max(axis=0)
img_corr = (Cn-np.mean(Cn))/np.std(Cn)
summary_images = np.stack([img, img, img_corr], axis=0).astype(np.float32)
# Save summary images which could be further used in the VolPy GUI
cm.movie(summary_images).save(fnames[:-5] + '_summary_images.tif')

fig, axs = plt.subplots(1, 2)
axs[0].imshow(summary_images[0]); axs[1].imshow(summary_images[2])
axs[0].set_title('mean image'); axs[1].set_title('corr image')

Text(0.5, 1.0, 'corr image')

In [18]:
use_maskrcnn = True  # set to True to predict the ROIs using the mask R-CNN
if not use_maskrcnn:                 # use manual annotations
    with h5py.File(path_ROIs, 'r') as fl:
        ROIs = fl['mov'][()]  # load ROIs
else:
    weights_path = download_model('mask_rcnn')    
    ROIs = utils.mrcnn_inference(img=summary_images.transpose([1, 2, 0]), size_range=[5, 22],
                                  weights_path=weights_path, display_result=True) # size parameter decides size range of masks to be selected
    cm.movie(ROIs).save(fnames[:-5] + '_mrcnn_ROIs.hdf5')


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  128
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              pad64
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE               

Instructions for updating:
Use fn_output_signature instead


Processing 1 images
image                    shape: (180, 512, 3)         min:   -2.93339  max:    7.64126  float32
molded_images            shape: (1, 192, 512, 3)      min:   -2.93339  max:    7.64126  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  512.00000  int64
anchors                  shape: (1, 24552, 4)         min:   -0.94775  max:    1.61267  float32


/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [12]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(summary_images[0]); axs[1].imshow(ROIs.sum(0))
axs[0].set_title('mean image'); axs[1].set_title('masks')

Text(0.5, 1.0, 'masks')

In [27]:
# Restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=None, single_thread=False, maxtasksperchild=1)

Now processing cell number 2Now processing cell number 0Now processing cell number 1Now processing cell number 4Now processing cell number 3Now processing cell number 5Now processing cell number 6
Now processing cell number 7Now processing cell number 8
Now processing cell number 9Now processing cell number 10









/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)
/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)


/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)
/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)
/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and wi

Now processing cell number 11Now processing cell number 13Now processing cell number 12Now processing cell number 14Now processing cell number 15Now processing cell number 16
Now processing cell number 17

Now processing cell number 18


Now processing cell number 19

/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)


...

Now processing cell number 53



/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)
/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)


/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)
/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)
/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and wi

Now processing cell number 54


/home/janwillem/.conda/envs/caiman-env/lib/python3.9/site-packages/caiman/source_extraction/volpy/spikepursuit.py:194: FutureWarning: The parameters `shift_x` and `shift_y` are deprecated since v0.23 and will be removed in v0.26. Use `pad_footprint` or modify the footprintmanually instead.
  bwexp = dilation(bw, np.ones([args['context_size'], args['context_size']]), shift_x=True, shift_y=True)


## Trace denoising and spike extraction

In [8]:
m_orig.save('./test.tif')

'./test.tif'

In [23]:
test_tif = cm.load('./test.tif')

In [24]:
test_tif.shape

(7500, 116, 498)

In [39]:
ROIs = np.zeros((2, 180, 512), dtype='bool')
ROIs[0, 25:160, 20:480] = True
ROIs[1, 25:160, 20:480] = True

In [48]:
# Parameters for trace denoising and spike extraction
ROIs = ROIs                                   # region of interests
index = list(range(len(ROIs)))                # index of neurons
weights = None                                # if None, use ROIs for initialization; to reuse weights check reuse weights block 

template_size = 0.02[ 5  7  9 12 18 21 26 28 35 37 42 45 46 48 54]                          # half size of the window length for spike templates, default is 20 ms 
context_size = 60                             # number of pixels surrounding the ROI to censor from the background PCA
visualize_ROI = False                         # whether to visualize the region of interest inside the context region
flip_signal = True                            # Important!! Flip signal or not, True for Voltron indicator, False for others
hp_freq_pb = 1 / 3                            # parameter for high-pass filter to remove photobleaching
clip = 100                                    # maximum number of spikes to form spike template
threshold_method = 'adaptive_threshold'       # adaptive_threshold or simple 
min_spikes= 10                                # minimal spikes to be found
pnorm = 0.5                                   # a variable deciding the amount of spikes chosen for adaptive threshold method
threshold = 3                                 # threshold for finding spikes only used in simple threshold method, Increase the threshold to find less spikes
do_plot = False                               # plot detail of spikes, template for the last iteration
ridge_bg= 0.01                                # ridge regression regularizer strength for background removement, larger value specifies stronger regularization 
sub_freq = 20                                 # frequency for subthreshold extraction
weight_update = 'ridge'                       # ridge or NMF for weight update
n_iter = 2                                    # number of iterations alternating between estimating spike times and spatial filters
opts_dict={'fnames': fname_new,
            'ROIs': ROIs,
            'index': index,
            'weights': weights,
            'template_size': template_size, 
            'context_size': context_size,
            'visualize_ROI': visualize_ROI, 
            'flip_signal': flip_signal,
            'hp_freq_pb': hp_freq_pb,
            'clip': clip,
            'threshold_method': threshold_method,
            'min_spikes':min_spikes,
            'pnorm': pnorm, 
            'threshold': threshold,
            'do_plot':do_plot,
            'ridge_bg':ridge_bg,
            'sub_freq': sub_freq,
            'weight_update': weight_update,
            'n_iter': n_iter}

opts.change_params(params_dict=opts_dict);    

In [ ]:
vpy = VOLPY(n_processes=n_processes, dview=dview, params=opts)
vpy.fit(n_processes=n_processes, dview=dview)

## Visualization

In [ ]:
# Visualize spatial footprints and traces
print(np.where(vpy.estimates['locality'])[0])    # neurons that pass locality test
idx = np.where(vpy.estimates['locality'] > 0)[0]
print(idx)
utils.view_components(vpy.estimates, img_corr, idx)

(62, 7000)


In [47]:
# reconstructed movie
scope = (0,7000)

vpy.estimates['weights'][vpy.estimates['weights']<0] = 0    
A = vpy.estimates['weights'][idx].transpose([1,2,0]).reshape((-1,len(idx)))
C = vpy.estimates['t_rec'][idx,scope[0]:scope[1]]
mv_rec = np.dot(A, C).reshape((180,512,scope[1]-scope[0])).transpose((2,0,1))    
mv_rec = (mv_rec - mv_rec.min())/(mv_rec.max()-mv_rec.min())

mv_rec = cm.movie(mv_rec,fr=400)
mv_rec.play(fr=400)

0.00013454592996185234


In [44]:
# Reconstructed movie
mv_all = utils.reconstructed_movie(vpy.estimates.copy(), fnames=mc.mmap_file,
                                           idx=idx, scope=(0,1000), flip_signal=flip_signal)


mv_all.play(fr=40, magnification=3)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.25it/s]


In [18]:
vpy.estimates['ROIs'] = ROIs
save_name = f'volpy_{os.path.split(fnames)[1][:-5]}_{threshold_method}'
np.save(os.path.join(file_dir, save_name), vpy.estimates)

In [19]:
#%% Reuse spatial weights extracted from previous video
# set weights = reuse_weights in opts_dict dictionary
if False:
    estimates = np.load(os.path.join(file_dir, save_name+'.npy'), allow_pickle=True).item()
    reuse_weights = []
    for idx in range(ROIs.shape[0]):
        coord = estimates['context_coord'][idx]
        w = estimates['weights'][idx][coord[0][0]:coord[1][0]+1, coord[0][1]:coord[1][1]+1] 
        plt.figure(); plt.imshow(w);plt.colorbar(); plt.show()
        reuse_weights.append(w)

In [20]:
# Stop cluster and clean up log files
cm.stop_server(dview=dview)
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)